#### Update content of the CPM Atlas - analysis plots for diurnal cycles of precipitation<br>
Author          : Team BETA<br>
Return Values   : png files<br>
Source data     : The data is preprocessed and provided by Petter Lind.

In [1]:
import numpy as np
import xarray as xr
from pathlib import Path

In [2]:
# path to datasets
path_data = "/mnt/d/NLeSC/BETA/EUCP/Data_Catalogue/data_Petter_Lind/diurnal"

In [3]:
# define availability of models by region
models_avai_cpm = {
    'NW': {
        'CNRM': 'nwe-3_cnrm-arome_cnrm-cm5',
        'KNMI': 'alp-3_hclim-knmi_ec-earth',
        'ETHZ': 'reu-3_ethz-cclm_ecmwf-erai',
        'UKMO': 'reu-3_mohc-um10.1_hadgem3-gc3.1',
    },
    'SW': {
        'CMCC': 'swe-3_cclm_ec-earth',
        'IPSL': 'swe-3_ipsl-wrf_ipsl-cm6',
        'ETHZ': 'reu-3_ethz-cclm_ecmwf-erai',
        'UKMO': 'reu-3_mohc-um10.1_hadgem3-gc3.1'
    }
}

        # SE: ['ICTP', 'ETHZ', 'UKMO'],
        # C: ['GERICS', 'ETHZ', 'UKMO'],
        # CE: ['SMHI', 'ICTP', 'ETHZ', 'UKMO'],
        # N: ['SMHI', 'GERICS'],
        # AL: ['CNRM', 'CMCC', 'IPSL', 'KNMI', 'GERICS', 'ETHZ', 'SMHI', 'ICTP', 'UKMO']

In [4]:
# define geometries for clipping by regions
regions = {
  "NW": [
    {
      'type': 'Polygon',
      'coordinates': [[[-8.0, 40.4], [11.0, 40.4], [15.2, 58.6], [-12.5, 58.6], [-8.0, 40.4]]]
    }
  ],
  "SW": [
    {
      'type': 'Polygon',
      'coordinates': [[[-10, 30], [7.4, 33], [5.7, 48.9], [-15, 45.4], [-10, 30]]]
    }
  ],
  "SE": [
    {
      'type': 'Polygon',
      'coordinates': [[[12.5, 34.3], [28.5, 34.3], [29.4, 40.9], [11.5, 40.9], [12.5, 34.3]]]
    }
  ],
  "C": [
    {
      'type': 'Polygon',
      'coordinates': [[[5.0, 44.5], [18.0, 45.5], [18.0, 56.0], [1.0, 53.0], [5.0, 44.5]]]
    }
  ],
  "CE": [
    {
      'type': 'Polygon',
      'coordinates': [[[17.8, 41.5], [31.3, 41.5], [32.8, 51.6], [16.4, 51.6], [17.8, 41.5]]]
    }
  ],
  "N": [
    {
      'type': 'Polygon',
      'coordinates': [[[1, 50.7], [26.7, 49.7], [44.1, 70.6], [-9.4, 72.6], [1, 50.7]]]
    }
  ],
  "AL": [
    {
      'type': 'Polygon',
      'coordinates': [[[1, 40], [17, 40], [17, 50], [1, 50], [1, 40]]]
    }
  ]
}

In [5]:
# choose experiment
experiment = "rcp85"
#projects = ["CPM", "RCM"]
projects = ["CPM"]

In [6]:
for region_alias in regions.keys():
    print(region_alias)
    # create empty dict to host the data
    precipitation_diurnal_cpm = {}
    precipitation_diurnal_rcm = {}
    for project in projects:
        directory = path_data + "/" + project
        for model in models_avai_cpm[f'{region_alias}'].keys():
            model_file = models_avai_cpm[f'{region_alias}'][f'{model}']
            ncfile = [data_file for data_file in Path(directory).iterdir()
             if(model_file in data_file.stem and experiment in data_file.stem)]
            print(ncfile)
            # clipped_region = region_clip(data, region_alias)
        

[PosixPath('/mnt/d/NLeSC/BETA/EUCP/Data_Catalogue/data_Petter_Lind/diurnal/CPM/nwe-3_cnrm-arome_cnrm-cm5_rcp85_diurnal_cycle_amount_pr_1hr_mean_native_grid_2041-2050_JJA.nc')]
[PosixPath('/mnt/d/NLeSC/BETA/EUCP/Data_Catalogue/data_Petter_Lind/diurnal/CPM/alp-3_hclim-knmi_ec-earth_rcp85_diurnal_cycle_amount_pr_1hr_mean_native_grid_2041-2050_JJA.nc')]
[PosixPath('/mnt/d/NLeSC/BETA/EUCP/Data_Catalogue/data_Petter_Lind/diurnal/CPM/reu-3_ethz-cclm_ecmwf-erai_rcp85_diurnal_cycle_amount_pr_1hr_mean_native_grid_2080-2089_JJA.nc')]
[PosixPath('/mnt/d/NLeSC/BETA/EUCP/Data_Catalogue/data_Petter_Lind/diurnal/CPM/reu-3_mohc-um10.1_hadgem3-gc3.1_rcp85_diurnal_cycle_amount_pr_1hr_mean_native_grid_2040-2049_JJA.nc')]
[PosixPath('/mnt/d/NLeSC/BETA/EUCP/Data_Catalogue/data_Petter_Lind/diurnal/CPM/swe-3_cclm_ec-earth_rcp85_diurnal_cycle_amount_pr_1hr_mean_native_grid_2041-2050_JJA.nc')]
[PosixPath('/mnt/d/NLeSC/BETA/EUCP/Data_Catalogue/data_Petter_Lind/diurnal/CPM/swe-3_ipsl-wrf_ipsl-cm6_rcp85_diurnal_cy

KeyError: 'SE'